This notebook creates Figure 4 for a 3 selected scenarios. Requirement: the specified scenarios must have been calculated and stored in the "results_meoh_case.xlsx"

In [ ]:
scenarios = ['Base', 'NDC', 'PkBudg500']
year = 2040

### Import Section

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import matplotlib 
plt.rcParams["figure.figsize"] = [10, 4.00]
plt.rcParams["figure.autolayout"] = True

### Define colors

In [ ]:
cmap = np.concatenate([['#D2D2D3'], ['#4492f4'],['#f84540']])

### Prepare Data

In [ ]:
df = pd.read_excel('Input/results_meoh_case.xlsx')

In [ ]:
def interpolate_eps(scenario, year):
    # reduce file to the selected scenario / year
    filtered_df = df[(df['scenario'] == scenario) & (df['year'] == int(year))]
    # find all the "BAU" rows, there the GWP reduction data is storred
    filtered_df2 = filtered_df[filtered_df['net-zero GWP [Mt]'] != 0]
    # read the gwp reduction target
    net_zero_gwp = max(filtered_df2['net-zero GWP [Mt]'])
    # filter the data to those there the reduction potential is smaller or bigger than the target
    filtered_df3 = filtered_df2[filtered_df2['GWP reduction [Mt]'] < net_zero_gwp]
    filtered_df4 = filtered_df2[filtered_df2['GWP reduction [Mt]'] > net_zero_gwp]
    # read the closest values to the target
    lower = max(filtered_df3['GWP reduction [Mt]'])
    upper = min(filtered_df4['GWP reduction [Mt]'])
    # get the associated epsilon values
    lower_eps = max(filtered_df3['epsilon [%]'])
    upper_eps = min(filtered_df4['epsilon [%]'])
    # find the epsilon which linearily interpolates between the two closest solutions and the target
    eps = (net_zero_gwp - lower)/(upper - lower)*(upper_eps-lower_eps) + lower_eps
    # Use this epsilon to interpolate all the production data results
    df1 = filtered_df[filtered_df['epsilon [%]'] == lower_eps]
    df2 = filtered_df[filtered_df['epsilon [%]'] == upper_eps]
    # Create a new DataFrame for interpolated data
    interpolated_rows = []

    # Iterate through each row and interpolate the values based on epsilon
    for index, row1 in df1.iterrows():
        df2red = df2[(df2['region'] == row1['region']) & (df2['tech'] == row1['tech'])]
        for index2, row2 in df2red.iterrows():
            if row1['epsilon [%]'] == eps:
                # No interpolation needed, just append the row from df1
                interpolated_rows.append(row1)
            else:
                # Linear interpolation
                alpha = (eps - lower_eps) / (upper_eps - lower_eps)
                interpolated_values = {}
                for column in df1.columns:
                    if column not in ['scenario', 'year', 'epsilon [%]', 'region', 'tech']:
                        interpolated_values[column] = alpha * row2[column] + (1 - alpha) * row1[column]
                    elif column == 'epsilon [%]':
                        interpolated_values[column] = eps
                    else:
                        interpolated_values[column] = row1[column]
                interpolated_rows.append(interpolated_values)

    # Create the interpolated DataFrame using pandas.concat
    interpolated_df = pd.concat([pd.DataFrame([row]) for row in interpolated_rows], ignore_index=True)
    return interpolated_df

Interpolate between the three selected scenarios

In [ ]:
dfs = {scenario: interpolate_eps(scenario, year) for scenario in scenarios}

In [ ]:
summed_dfs = []
for scen in dfs:
    new_df = dfs[scen][['tech', 'meoh [Mt]']]
    summed_dfs.append(new_df.groupby('tech')['meoh [Mt]'].sum().reset_index().set_index('tech').rename(columns={'meoh [Mt]': scen}))

In [ ]:
df_concat = pd.concat(summed_dfs, ignore_index=False, axis=1).transpose()
df_concat = df_concat[['BAU', 'PSC', 'DAC']]

In [ ]:
elec_scatter = [max(dfs[scen]['elec [TWh]']) for scen in dfs]

### Plotting function 🎨

In [ ]:
font = {'size': 16}

matplotlib.rc('font', **font)

fig, ax = plt.subplots()
df_concat.plot.bar(stacked=True,legend=False, edgecolor='black', color=cmap, ax=ax)
plt.axhline(y=0,linewidth=0.5, color='black')

# Additional Electricity
ax2 = ax.twinx() 
ax2.bar((0.31,1.31,2.31), elec_scatter, edgecolor='black', facecolor='orange',width = 0.1)

ax.set_ylabel("CCU methanol production [Mt]")

ax.set_ylim([0,155])
ax.set_xlim([-0.5,2.5])
ax.set_xticklabels(['Base  (3.5ºC)', 'NDC (2ºC)', 'PkBudg500 (1.5ºC)'], rotation = 0)
ax.set_yticks([0,20,40,60,80,100],[0,20,40,60,80,100])

ax2.set_ylim([0,600])
ax2.set_xlim([-0.5,2.5])
ax2.set_ylabel('Δ Electricity [TWh]')
ax2.spines['right'].set_color('darkorange')
ax2.tick_params(axis='y', colors='darkorange')

DAC_patch = patches.Patch(edgecolor='black', facecolor='#f84540', label='Direct air capture')
PS_patch = patches.Patch(edgecolor='black', facecolor='#4492f4', label='Point-source capture')
BAU_patch = patches.Patch(edgecolor='black', facecolor='#D2D2D3', label='BAU')
legend1 = plt.legend(handles=[DAC_patch, PS_patch, BAU_patch], ncol=2, bbox_to_anchor=(0.38, 0.7))

fig.tight_layout()

plt.savefig('Output/Figure 4 ' + str(year), dpi=300)

In [ ]:
elec_scatter

In [ ]:
df_concat.to_excel('temp.xlsx')